In [1]:
!pip install gmaps

In [24]:
# VacationPy
#----

#### Note
#* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

In [8]:
vaca_csv="output_data/cities.csv"
vaca = pd.read_csv(vaca_csv)
vaca.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [9]:
vaca.nunique()

City_ID       548
City          548
Cloudiness     86
Country       123
Date          175
Humidity       89
Lat           532
Lng           543
Max Temp      261
Wind Speed    251
dtype: int64

In [10]:
hum = vaca[['Humidity', 'Lat', 'Lng']]
hum

,Humidity,Lat,Lng
0,72,68.05,39.51
1,74,-20.41,57.70
2,73,77.48,-69.36
3,39,40.71,112.04
4,76,-33.93,18.42
...,...,...,...
543,32,33.97,36.66
544,4,24.01,-104.61
545,62,49.65,0.71
546,93,51.07,132.56


In [11]:
city = vaca[['City','Humidity', 'Lat', 'Lng']]
city.drop_duplicates('City')

,City,Humidity,Lat,Lng
0,ostrovnoy,72,68.05,39.51
1,mahebourg,74,-20.41,57.70
2,qaanaaq,73,77.48,-69.36
3,zhuhai,39,40.71,112.04
4,cape town,76,-33.93,18.42
...,...,...,...,...
543,yabrud,32,33.97,36.66
544,paraiso,4,24.01,-104.61
545,veraval,62,49.65,0.71
546,novyy urgal,93,51.07,132.56


In [12]:
### Humidity Heatmap
#* Configure gmaps.
#* Use the Lat and Lng as locations and Humidity as the weight.
#* Add Heatmap layer to map.
figure_layout = {
    'width': '1300px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}

#configure
gmaps.configure(g_key)

#identify
locations = vaca[["Lat", "Lng"]]
weight = vaca["Humidity"]

#plot heatmap
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,30))
heat = gmaps.heatmap_layer(locations, weights=weight,
                           dissipating=True, max_intensity=10,
                                 point_radius=5)
#add layer
fig.add_layer(heat)
#display fig
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='2px', wi…

Create new DataFrame fitting weather criteria
    Narrow down the cities to fit weather conditions.
    Drop any rows will null values.

In [13]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

data= vaca[["City", "Max Temp", "Wind Speed", "Cloudiness"]]
data
temp=(data['Max Temp'] <=80) & (data['Max Temp'] > 70)
wind= data['Wind Speed'] < 10
cloud = data['Cloudiness'] == 0
data[temp & wind & cloud]

,City,Max Temp,Wind Speed,Cloudiness
7,opuwo,72.06,5.44,0
70,ponta do sol,73.68,5.32,0
88,cidreira,76.56,5.50,0
138,morondava,79.08,9.06,0
139,vaini,78.90,1.59,0
176,nador,75.20,6.93,0
297,mogok,72.24,2.77,0
444,birjand,71.60,9.17,0


In [14]:
data1= vaca[["City", "Max Temp", "Wind Speed", "Cloudiness", "Lng","Lat","Country", "Humidity"]]
data1
temp1=(data1['Max Temp'] <=80) & (data['Max Temp'] > 70)
wind1= data1['Wind Speed'] < 10
cloud1 = data1['Cloudiness'] == 0
data1[temp1 & wind1 & cloud1]

,City,Max Temp,Wind Speed,Cloudiness,Lng,Lat,Country,Humidity
7,opuwo,72.06,5.44,0,13.84,-18.06,NaN,23
70,ponta do sol,73.68,5.32,0,-46.00,-20.63,BR,57
88,cidreira,76.56,5.50,0,-50.22,-30.17,BR,66
138,morondava,79.08,9.06,0,44.28,-20.30,MG,69
139,vaini,78.90,1.59,0,74.49,15.34,IN,88
176,nador,75.20,6.93,0,-2.93,35.17,MA,40
297,mogok,72.24,2.77,0,96.51,22.92,MM,51
444,birjand,71.60,9.17,0,59.22,32.86,IR,26


In [15]:
#configure
gmaps.configure(g_key)

#layout

figure_layout = {
    'width': '1300px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}

#identify
locations1 = data1[["Lat", "Lng"]]
weight1 = data1["Humidity"]

#plot heatmap
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,30))
heat = gmaps.heatmap_layer(locations= locations1, weights= weight1,
                           dissipating=True, max_intensity=10,
                                 point_radius=3)
#add layer
fig.add_layer(heat)
#display fig
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='2px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = data1[["City","Country","Lat", "Lng","Humidity"]]
#set up empty string / add empty column
#hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Humidity
0,ostrovnoy,RU,68.05,39.51,72
1,mahebourg,MU,-20.41,57.70,74
2,qaanaaq,GL,77.48,-69.36,73
3,zhuhai,CN,40.71,112.04,39
4,cape town,ZA,-33.93,18.42,76
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,32
544,paraiso,MX,24.01,-104.61,4
545,veraval,FR,49.65,0.71,62
546,novyy urgal,RU,51.07,132.56,93


In [17]:
#Coordinates
target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"

#set up a dictionary
params = {
    "location": target_coordinates,
    "keyword": 'hotel',
    "radius": 5000,
    "type": 'hotel',
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run a request
    response = requests.get(base_url, params=params)

    response = response.json()

    results= response['results']

    try:
        print(f"\tThe closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (IndexError):
        print("Missing field/result... skipping.")

tel Motel.
	The closest hotel is Sur Plaza Hotel.
Missing field/result... skipping.
	The closest hotel is Emerald Star Hotel.
	The closest hotel is Mercure Launceston.
	The closest hotel is Tourist House - Sablino.
	The closest hotel is Hotel National.
	The closest hotel is Pond Country Club.
Missing field/result... skipping.
	The closest hotel is Hotel Ferreira.
	The closest hotel is Hotel Río Arriba.
	The closest hotel is Meili Shanshui Hotel.
	The closest hotel is Best Western Plus Delta Inn & Suites.
	The closest hotel is Park Inn by Radisson Najran.
Missing field/result... skipping.
	The closest hotel is Southland Organic Farmstay.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
	The closest hotel is Swiss-Belhotel Sorong.
	The closest hotel is Four Points by Sheraton Kingston.
	The closest hotel is The Landing Hotel.
	The closest hotel is Hilton Garden Inn Kauai Wailua Bay.
Missing field/result... skipping.
	The closest hotel 

In [18]:
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 27.730232, 'lng': 109.18133},
   'viewport': {'northeast': {'lat': 27.73152932989272,
     'lng': 109.1826955798927},
    'southwest': {'lat': 27.72882967010728, 'lng': 109.1799959201073}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '14ca1146d42b71cd0e7f7907fdfa204b778d2b46',
  'name': 'Yudu Hotel',
  'photos': [{'height': 2240,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112938191641416269920">Voon Leong Lim</a>'],
    'photo_reference': 'CmRaAAAA3mwica780hs9JwprL2-y6zTNoKTtPCdpHU98KdzEUJWLBjL89eCTTx_imYMQGBHDUA72lNm1bD8DTaLZDIU7d4b67NHERL_EvUnxqQolmZv9lKCGcc-2lkVVjz04qjyPEhCnRWTwd0oBq4RF1b84hh2IGhRT2tnX__AelIvOk5bdxX7Sw2Nrtg',
    'width': 3968}],
  'place_id': 'ChIJexeBty6-lzYRPuSs3r33a58',
  'plus_code': {'compound_code': 'P5JJ+3G Bijiang District, Tongren, Guizhou, China',
   'global_code': '7PVFP5JJ+3G'},
  'rating': 5,
  'reference': 'Ch

In [19]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ostrovnoy,RU,68.05,39.51,72,NaN
1,mahebourg,MU,-20.41,57.70,74,Here paradise hotel
2,qaanaaq,GL,77.48,-69.36,73,Qaanaaq Hotel
3,zhuhai,CN,40.71,112.04,39,NaN
4,cape town,ZA,-33.93,18.42,76,Atlanticview Cape Town Boutique Hotel
...,...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,32,NaN
544,paraiso,MX,24.01,-104.61,4,Hampton Inn by Hilton Durango
545,veraval,FR,49.65,0.71,62,B&B HOTEL
546,novyy urgal,RU,51.07,132.56,93,NaN


In [20]:
#hotel_df.to_csv('hotels.csv')

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='2px', wi…